In [ ]:
import copy
import os
import numpy as np
import random
import time
import copy
from operator import attrgetter

import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from enum import Enum
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import keras
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Flatten, Activation, Conv2D, MaxPooling2D, AveragePooling2D, Dropout
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam

# Configuration

In [ ]:
class ActivationType(Enum):
  relu = 'relu'
  softmax = 'softmax'     
  elu = 'elu'  

In [ ]:
class PoolingTypes(Enum):
  avg_pooling = 1
  max_pooling = 2

In [ ]:
class OptimizerType(Enum):
  adam = 1
  sgd = 2

In [ ]:
class Configuration:
  kernel_size_set = range(1, 10, 2)
  act_set = [ActivationType.relu, ActivationType.softmax]
  pooling_set = [PoolingTypes.max_pooling, PoolingTypes.avg_pooling]
  neuron_set = range(1, 300)
  epochs_set = range(1, 20)  
  batch_size_set = [16, 32, 64]
  optimizer_set = [OptimizerType.adam, OptimizerType.sgd]

  def __init__(self, convulution_kernel_size = 3, 
               convulution_act = ActivationType.relu, 
               pooling_layer_type = PoolingTypes.max_pooling, 
               pooling_layer_kernel_size = 2, 
               first_dense_layer_neurons = 256, 
               first_dense_act = ActivationType.relu, 
               second_dense_act = ActivationType.softmax, 
               include_first_droput_layer = False,
               first_dropout_rate = 0.5, 
               include_second_droput_layer = True,
               second_dropout_rate = 0.5, 
               optimizer_type = OptimizerType.sgd, 
               learning_rate = 0.01,
               epochs = 15, 
               batch_size = 32) -> None:
    self.convulution_kernel_size = convulution_kernel_size
    self.convulution_act = convulution_act
    self.pooling_layer_type = pooling_layer_type
    self.pooling_layer_kernel_size = pooling_layer_kernel_size
    self.first_dense_layer_neurons = first_dense_layer_neurons
    self.first_dense_act = first_dense_act    
    self.second_dense_act = second_dense_act
    self.include_first_droput_layer = include_first_droput_layer
    self.first_dropout_rate = first_dropout_rate
    self.include_second_droput_layer = include_second_droput_layer
    self.second_dropout_rate = second_dropout_rate
    self.optimizer_type = optimizer_type
    self.learning_rate = learning_rate
    self.epochs = epochs
    self.batch_size = batch_size
    self.loss = None
    self.fitness = 0

In [ ]:
def create_random():
  first_dense_neurons = random.choice(Configuration.neuron_set)
  second_dense_neurons = random.choice(range(min(Configuration.neuron_set), first_dense_neurons))
  return Configuration(
      random.choice(Configuration.kernel_size_set),
      random.choice(Configuration.act_set),
      random.choice(Configuration.pooling_set),
      random.choice(Configuration.kernel_size_set),
      first_dense_neurons,
      random.choice(Configuration.act_set),
      random.choice(Configuration.act_set),
      bool(random.getrandbits(1)),
      random.uniform(0, 1),
      bool(random.getrandbits(1)),
      random.uniform(0, 1),
      random.choice(Configuration.optimizer_set),
      random.uniform(0.0001, 0.2),
      random.choice(Configuration.epochs_set),
      random.choice(Configuration.batch_size_set)
  )

# CNN

In [ ]:
def plotLosses(history):  
    plt.plot(history.history['loss'])
    # plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
def cnn(config, img_train, img_test, label_train, label_test):
  if config.loss is not None:
    return config.loss

  model = Sequential()

  model.add(Conv2D(32, (config.convulution_kernel_size, config.convulution_kernel_size), activation=config.convulution_act.value, input_shape=(32, 32, 3)))
  model.add(Conv2D(32, (config.convulution_kernel_size, config.convulution_kernel_size), activation=config.convulution_act.value))

  if config.pooling_layer_type == PoolingTypes.max_pooling:
    model.add(MaxPooling2D(pool_size=(config.pooling_layer_kernel_size, config.pooling_layer_kernel_size)))
  elif config.pooling_layer_type == PoolingTypes.avg_pooling:
    model.add(AveragePooling2D(pool_size = (config.pooling_layer_kernel_size, config.pooling_layer_kernel_size)))
  if config.include_first_droput_layer:
    model.add(Dropout(config.first_dropout_rate))

  model.add(Flatten())
  model.add(Dense(config.first_dense_layer_neurons, activation=config.first_dense_act.value))
  if config.include_second_droput_layer:
    model.add(Dropout(config.second_dropout_rate))

  model.add(Dense(10, activation=config.second_dense_act.value)) # ennek softmaxnak kell lennie

  if config.optimizer_type == OptimizerType.sgd:
    opt = SGD(lr=config.learning_rate, decay=1e-6, momentum=0.9, nesterov=True)
  elif config.optimizer_type == OptimizerType.adam:
    opt = Adam(lr=config.learning_rate)

  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=opt)
  history = model.fit(img_train,label_train, epochs=config.epochs, batch_size=config.batch_size, verbose=2, validation_split=0.0)
  #plotLosses(history)
  return model.evaluate(img_test, label_test)[0]


In [ ]:
(img_train, label_train), (img_test, label_test) = cifar10.load_data()
label_train = to_categorical(label_train, num_classes=10)
label_test = to_categorical(label_test, num_classes=10)
img_train = img_train.astype('float32')
img_test = img_test.astype('float32')
img_train /= 255
img_test /= 255

170508288/170498071 [==============================] - 3s 0us/step


In [ ]:
config = Configuration(2, ActivationType.relu, PoolingTypes.avg_pooling, 2, 151, ActivationType.relu, 42, ActivationType.softmax, 0.6599541350938929, OptimizerType.sgd, 0.020280410342938485, 3, 32)
res = cnn(config, img_train, img_test, label_train, label_test)

print(res)

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


Epoch 1/3
 - 24s - loss: 1.5788 - acc: 0.4327
Epoch 2/3
 - 15s - loss: 1.2063 - acc: 0.5717
Epoch 3/3
 - 15s - loss: 0.9899 - acc: 0.6492
10000/10000 [==============================] - 4s 418us/step
1.0704704772949218


# Genetic Algorithm

In [ ]:
class GA_parameters:
  max_population_size = 50
  min_population_size = 6
  population_size = 40
  max_number_of_generations = 90
  min_number_of_generations = 10
  number_of_generations = 25
  mutation_rate = 0.3
  min_mutation_rate = 0.0
  crossover_rate = 0.8
  min_crossover_rate = 0.1
  parameters_number = 15
  elite_rate = 0.1

In [ ]:
def fitness(population):
  loss_values = np.empty((len(population), 2), float)
  
  fitness_values = [0 for _ in range(len(population))]
  for j in range(len(population)):
    loss_values[j][0] = j
    loss_values[j][1] = population[j].loss

  loss_sorted = loss_values[loss_values[:, 1].argsort()]

  sp = 2#loss_sorted[0][1] - loss_sorted[-1][1]
  for j in range(len(population), 0, -1):
    fitness = 2-sp + 2 * (sp - 1) * (j - 1) / (len(population) - 1)
    fitness_values[int(loss_sorted[len(population)-j][0])] = fitness
    population[int(loss_sorted[len(population)-j][0])].fitness = fitness

  return fitness_values

In [ ]:
def select_parents(fitness_values, population):
  selected_parent_idx = [0 for _ in range(GA_parameters.population_size)]
  fitness_tuples = [(i, fitness_values[i]) for i in range(GA_parameters.population_size)]
  sorted_fitnesses = sorted(fitness_tuples, key=lambda tup: tup[1])
  print(sorted_fitnesses)
  #sorted_fitnesses = sorted_fitnesses[::-1]
  print(sorted_fitnesses)
  fitness_sum = sum(fitness_values)
  distance = fitness_sum / GA_parameters.population_size
  shift = random.uniform(0, distance)

  borders = [0 for _ in range(GA_parameters.population_size)]

  for i in range(GA_parameters.population_size):
    new_border = shift + i * distance
    borders[i] = new_border

  for border_id in range(len(borders)):
    i = 0
    roulette_sum = sorted_fitnesses[i][1]
    while roulette_sum < borders[border_id]:
      i = i + 1
      roulette_sum = roulette_sum + sorted_fitnesses[i][1]

    selected_parent_idx[border_id] = sorted_fitnesses[i][0]

  return selected_parent_idx

In [ ]:
def crossover_blend(p1, p2, round_value):
    c1 = p1
    c2 = p2

    r = random.uniform(0, 1)
    c1 = p1 + r * (p2 - p1)
    c2 = p2 - r * (p2 - p1)

    if round_value:
      c1 = round(c1)
      c2 = round(c2)

    return c1, c2


In [ ]:
def crossover_operation(p1, p2):
    prob = 0.5
    c1 = copy.deepcopy(p1)
    c2 = copy.deepcopy(p2)

    if random.uniform(0,1) < prob:
      c1.convulution_kernel_size, c2.convulution_kernel_size = crossover_blend(p1.convulution_kernel_size, p2.convulution_kernel_size, True)
    if random.uniform(0,1) < prob:
      c1.convulution_act, c2.convulution_act = p2.convulution_act, p1.convulution_act
    if random.uniform(0,1) < prob:
      c1.pooling_layer_type, c2.pooling_layer_type = p2.pooling_layer_type, p1.pooling_layer_type
    if random.uniform(0,1) < prob:
      c1.pooling_layer_kernel_size, c2.pooling_layer_kernel_size = crossover_blend(p1.pooling_layer_kernel_size, p2.pooling_layer_kernel_size, True)
    if random.uniform(0,1) < prob:
      c1.first_dense_layer_neurons, c2.first_dense_layer_neurons = crossover_blend(p1.first_dense_layer_neurons, p2.first_dense_layer_neurons, True)
    if random.uniform(0,1) < prob:
      c1.first_dense_act, c2.first_dense_act = p2.first_dense_act, p1.first_dense_act
    if random.uniform(0,1) < prob:
      c1.second_dense_layer_neurons, c2.second_dense_layer_neurons = crossover_blend(p1.second_dense_layer_neurons, p2.second_dense_layer_neurons, True)
    if random.uniform(0,1) < prob:
      c1.second_dense_act, c2.second_dense_act = p2.second_dense_act, p1.second_dense_act
    if random.uniform(0,1) < prob:
      c1.first_dropout_rate, c2.first_dropout_rate = crossover_blend(p1.first_dropout_rate, p2.first_dropout_rate, False)
    if random.uniform(0,1) < prob:
      c1.second_dropout_rate, c2.second_dropout_rate = crossover_blend(p1.second_dropout_rate, p2.second_dropout_rate, False)
    if random.uniform(0,1) < prob:
      c1.include_first_droput_layer, c2.include_first_droput_layer = p2.include_first_droput_layer, p1.include_first_droput_layer
    if random.uniform(0,1) < prob:
      c1.include_second_droput_layer, c2.include_second_droput_layer = p2.include_second_droput_layer, p1.include_second_droput_layer
    if random.uniform(0,1) < prob:
      c1.optimizer_type, c2.optimizer_type = p2.optimizer_type, p1.optimizer_type
    if random.uniform(0,1) < prob:
      c1.learning_rate, c2.learning_rate = crossover_blend(p1.learning_rate, p2.learning_rate, False)
    if random.uniform(0,1) < prob:
      c1.epochs, c2.epochs = crossover_blend(p1.epochs, p2.epochs, True)
    if random.uniform(0,1) < prob:
      c1.batch_size, c2.batch_size = p2.batch_size, p1.batch_size

    c1.fitness = 0
    c2.fitness = 0
    c1.loss = None
    c2.loss = None
    return c1, c2

In [ ]:
def pair_exists(pairs, candidate):
  for p in pairs:
    if p == candidate or p == candidate[::-1]:
      return True
  return False

def crossover(population, parents):
  parent_pair_number = round(len(population) * GA_parameters.crossover_rate)

  combinations = len(set(parents)) * (len(set(parents))-1)
  if combinations < parent_pair_number:
      parent_pair_number = combinations

  pairs = [[0, 0] for _ in range(parent_pair_number)]
  for i in range(parent_pair_number):
    a = random.choice(parents)
    leftover = parents.copy()
    leftover = list(filter(lambda x: x != a, leftover))

    b = random.choice(leftover)
    random_pair = [a, b]
    while random_pair[0] == random_pair[1] and pair_exists(pairs, random_pair):
      a = random.choice(parents)
      leftover = parents
      leftover = list(filter(lambda x: x != a, leftover))
      print(leftover)
      b = random.choice(leftover)
      random_pair = [a, b]

    pairs[i] = random_pair

  childs = []

  for p in pairs:
    c1, c2 = crossover_operation(population[p[0]], population[p[1]])
    childs.append(c1)
    childs.append(c2)

  return childs


In [ ]:
def mutate(x, sigma, round_value, bounds):
    res = x + random.gauss(0, sigma)
    if round_value:
        res = round(res)
    if max(bounds) > res > min(bounds):
        return res

    return x

In [ ]:
def mutation_operation(population):
    prob = 0.5
    choosen = random.sample(range(0, len(population)), round(len(population) * GA_parameters.mutation_rate))
    mutated = []
    for p_id in choosen:
        p = population[p_id]
        m = copy.deepcopy(p)

        if random.uniform(0,1) < prob:
          m.convulution_kernel_size = mutate(p.convulution_kernel_size, 1, True, Configuration.kernel_size_set)
        if random.uniform(0,1) < prob:
          m.convulution_act = random.choice(Configuration.act_set)
        if random.uniform(0,1) < prob:
          m.pooling_layer_type = random.choice(Configuration.pooling_set)
        if random.uniform(0,1) < prob:
          m.pooling_layer_kernel_size = mutate(p.pooling_layer_kernel_size, 1, True, Configuration.kernel_size_set)
        if random.uniform(0,1) < prob:
          m.first_dense_layer_neurons = mutate(p.first_dense_layer_neurons, 10, True, Configuration.neuron_set)
        if random.uniform(0,1) < prob:
          m.first_dense_act = random.choice(Configuration.act_set)
        if random.uniform(0,1) < prob:
          m.second_dense_layer_neurons = mutate(p.second_dense_layer_neurons, 10, True, Configuration.neuron_set)
        if random.uniform(0,1) < prob:
          m.second_dense_act = random.choice(Configuration.act_set)
        if random.uniform(0,1) < prob:
          m.first_dropout_rate = mutate(p.first_dropout_rate, 0.05, False, [0, 1])
        if random.uniform(0,1) < prob:
          m.second_dropout_rate = mutate(p.second_dropout_rate, 0.05, False, [0, 1])
        if random.uniform(0,1) < prob:
          m.include_first_droput_layer = not m.include_first_droput_layer
        if random.uniform(0,1) < prob:
          m.include_second_droput_layer = not m.include_second_droput_layer
        if random.uniform(0,1) < prob:
          m.optimizer_type = random.choice(Configuration.optimizer_set)
        if random.uniform(0,1) < prob:
          m.learning_rate = mutate(p.learning_rate, 0.01, False, [0.0001, 0.1])
        if random.uniform(0,1) < prob:
          m.epochs = mutate(p.epochs, 1, True, [1, 20])
        if random.uniform(0,1) < prob:
          m.batch_size = random.choice(Configuration.batch_size_set)

        m.fitness = 0
        m.loss = None
        mutated.append(m)
    return mutated

In [31]:
def select_new_population(population, childs, mutated):
    sorted_pop = sorted(population, key=lambda x: x.fitness, reverse=True)
    new_population = sorted_pop[0 : round(len(population) * GA_parameters.elite_rate)]
    leftover = sorted_pop[round(len(population) * GA_parameters.elite_rate)::]

    leftover = leftover + childs + mutated

    def proc(i):
        loss = cnn(leftover[i], img_train, img_test, label_train, label_test)
        return loss

    losses = Parallel(n_jobs=None)(delayed(proc)(i) for i in range(len(leftover)))
    for j in range(len(leftover)):
      leftover[j].loss = losses[j]

    fitness_values = fitness(leftover)
    for j in range(len(leftover)):
      leftover[j].fitness = fitness_values[j]

    leftover_chosen = sorted(leftover, key=lambda x: x.fitness, reverse=True)[0 : GA_parameters.population_size - round(len(population) * GA_parameters.elite_rate)]

    new_population = new_population + leftover_chosen

    return new_population

In [ ]:
def average(series, period):
  if len(series) < period:
    return np.nan
  else:
    return sum(series[-period -1 : -1]) / period

In [ ]:
def is_improvement_positive(population_fit, period = 10, gap = 0.01):
  avg = average(population_fit, period)
  if np.isnan(avg) or avg == 0:
    return True

  return population_fit[-1] < avg * (1 - gap)

In [ ]:
population = [create_random() for a in range(GA_parameters.population_size)]
for j in range(GA_parameters.population_size):
  print(vars(population[j]))

def process(i):
    loss = cnn(population[i], img_train, img_test, label_train, label_test)
    #population[i].loss = loss
    return loss

start = time.perf_counter()
losses = Parallel(n_jobs=3)(delayed(process)(i) for i in range(GA_parameters.population_size))
end = time.perf_counter()
for j in range(GA_parameters.population_size):
  print(vars(population[j]))


In [ ]:
elapsed_time = end-start
print(f"{elapsed_time:0.4f} seconds")

1500.1381 seconds


In [ ]:
for j in range(GA_parameters.population_size):
  print(population[j].loss)

2.309623580169678
8.870509118652343
6.1983064155578615
2.3126065975189207
4.875065173339844
2.3059911834716798
11.613233776855468
7.52274270324707
2.309860329055786
2.304917765045166


In [ ]:
for j in range(GA_parameters.population_size):
  population[j].loss = losses[j]
  #print(vars(population[j]))

fitness_values = fitness(population)
for j in range(GA_parameters.population_size):
    population[j].fitness = fitness_values[j]

print(fitness_values)

[1.5555555555555556, 0.2222222222222222, 0.6666666666666666, 1.1111111111111112, 0.8888888888888888, 1.7777777777777777, 0.0, 0.4444444444444444, 1.3333333333333333, 2.0]


In [ ]:
parents = select_parents(fitness_values, population)

[(6, 0.0), (1, 0.2222222222222222), (7, 0.4444444444444444), (2, 0.6666666666666666), (4, 0.8888888888888888), (3, 1.1111111111111112), (8, 1.3333333333333333), (0, 1.5555555555555556), (5, 1.7777777777777777), (9, 2.0)]
[(6, 0.0), (1, 0.2222222222222222), (7, 0.4444444444444444), (2, 0.6666666666666666), (4, 0.8888888888888888), (3, 1.1111111111111112), (8, 1.3333333333333333), (0, 1.5555555555555556), (5, 1.7777777777777777), (9, 2.0)]


In [ ]:
childs = crossover(population, parents)
print(vars(childs[0]))

{'convulution_kernel_size': 3, 'convulution_act': <ActivationType.relu: 'relu'>, 'pooling_layer_type': <PoolingTypes.max_pooling: 2>, 'pooling_layer_kernel_size': 1, 'first_dense_layer_neurons': 207, 'first_dense_act': <ActivationType.relu: 'relu'>, 'second_dense_layer_neurons': 129, 'second_dense_act': <ActivationType.softmax: 'softmax'>, 'dropout_rate': 0.6897607009757344, 'optimizer_type': <OptimizerType.adam: 1>, 'learning_rate': 0.0951055041134371, 'epochs': 3, 'batch_size': 16, 'loss': 2.309623580169678, 'fitness': 1.5555555555555556}


In [ ]:
mutated = mutation_operation(population)
print(vars(mutated[0]))

{'convulution_kernel_size': 3, 'convulution_act': <ActivationType.relu: 'relu'>, 'pooling_layer_type': <PoolingTypes.avg_pooling: 1>, 'pooling_layer_kernel_size': 2, 'first_dense_layer_neurons': 133, 'first_dense_act': <ActivationType.relu: 'relu'>, 'second_dense_layer_neurons': 126, 'second_dense_act': <ActivationType.relu: 'relu'>, 'dropout_rate': 0.39073201474356, 'optimizer_type': <OptimizerType.adam: 1>, 'learning_rate': 0.053857042637628076, 'epochs': 4, 'batch_size': 16, 'loss': None, 'fitness': 0}


In [ ]:
new_population = select_new_population(population, childs, mutated)
print(new_population)

UnboundLocalError: local variable 'opt' referenced before assignment

In [ ]:
def get_gen_losses(population):
  result = []
  for i in population:
    result.append(i.loss)
  return result

In [34]:
best_config_in_gen = []
best_loss_in_gen = []
losses_in_gen = []
impr_list = []

def main(population):
    population = [create_random() for a in range(GA_parameters.population_size)]
    for j in range(GA_parameters.population_size):
        print(vars(population[j]))

    def process(i):
      loss = cnn(population[i], img_train, img_test, label_train, label_test)
      #population[i].loss = loss
      return loss

    start = time.perf_counter()
    losses = Parallel(n_jobs=None)(delayed(process)(i) for i in range(GA_parameters.population_size))
    end = time.perf_counter()
    for j in range(GA_parameters.population_size):
        print(vars(population[j]))

    elapsed_time = end-start
    print(f"{elapsed_time:0.4f} seconds")

    for j in range(GA_parameters.population_size):
        population[j].loss = losses[j]
        #print(vars(population[j]))
    #best_config_in_gen.append(min(population, key=attrgetter('loss')))
    #best_loss_in_gen.append(min(population, key=attrgetter('loss').loss))
    #losses_in_gen.append(get_gen_losses(population))
    gen = 0
    #for gen in range(GA_parameters.number_of_generations):
    while gen < GA_parameters.min_number_of_generations or (gen < GA_parameters.max_number_of_generations and is_improvement_positive(best_loss_in_gen, 10, 0.01)):
        print(gen)        
        impr_rate = is_improvement_positive(best_loss_in_gen, 5, 0.01)
        impr_list.append(impr_rate)

        if not impr_rate:
          GA_parameters.crossover_rate = min(GA_parameters.crossover_rate * 1.1, 1)
          GA_parameters.mutation_rate = min(GA_parameters.mutation_rate * 1.1, 1)
          if GA_parameters.population_size < GA_parameters.max_population_size:
            GA_parameters.population_size = GA_parameters.population_size + 2
            for _ in range(2):
              new_ind = create_random()
              new_ind.loss = cnn(new_ind, img_train, img_test, label_train, label_test)
              population.append(new_ind)
        else:   
          GA_parameters.crossover_rate = max(GA_parameters.crossover_rate * 0.9, GA_parameters.min_crossover_rate)
          GA_parameters.mutation_rate = max(GA_parameters.mutation_rate * 0.9, GA_parameters.min_mutation_rate)
          if GA_parameters.population_size > GA_parameters.min_population_size:
            GA_parameters.population_size = GA_parameters.population_size - 1
            worst_ind = max(population, key=attrgetter('loss'))
            population.remove(worst_ind)

        fitness_values = fitness(population)

        best_config_in_gen.append(min(population, key=attrgetter('loss')))
        best_loss_in_gen.append(min(population, key=attrgetter('loss')).loss)
        losses_in_gen.append(get_gen_losses(population))

        for j in range(GA_parameters.population_size):
            population[j].fitness = fitness_values[j]
        print('parents')
        parents = select_parents(fitness_values, population)
        print('childs')
        childs = crossover(population, parents)
        print('mutated')
        mutated = mutation_operation(population)
        print('select_new_population')
        population = select_new_population(population, childs, mutated)
        gen = gen + 1

    return population

In [ ]:
last_pop = main([])

{'convulution_kernel_size': 5, 'convulution_act': <ActivationType.softmax: 'softmax'>, 'pooling_layer_type': <PoolingTypes.avg_pooling: 1>, 'pooling_layer_kernel_size': 5, 'first_dense_layer_neurons': 163, 'first_dense_act': <ActivationType.relu: 'relu'>, 'second_dense_act': <ActivationType.relu: 'relu'>, 'include_first_droput_layer': True, 'first_dropout_rate': 0.021649993251133925, 'include_second_droput_layer': True, 'second_dropout_rate': 0.5385555102285889, 'optimizer_type': <OptimizerType.adam: 1>, 'learning_rate': 0.07907786817853395, 'epochs': 11, 'batch_size': 16, 'loss': None, 'fitness': 0}
{'convulution_kernel_size': 9, 'convulution_act': <ActivationType.relu: 'relu'>, 'pooling_layer_type': <PoolingTypes.avg_pooling: 1>, 'pooling_layer_kernel_size': 9, 'first_dense_layer_neurons': 124, 'first_dense_act': <ActivationType.softmax: 'softmax'>, 'second_dense_act': <ActivationType.softmax: 'softmax'>, 'include_first_droput_layer': True, 'first_dropout_rate': 0.8738529506508552, '

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/11
3125/3125 - 9s - loss: 10.2312 - accuracy: 0.0999 - 9s/epoch - 3ms/step
Epoch 2/11
3125/3125 - 8s - loss: 10.2274 - accuracy: 0.1008 - 8s/epoch - 3ms/step
Epoch 3/11
3125/3125 - 9s - loss: 9.2481 - accuracy: 0.0998 - 9s/epoch - 3ms/step
Epoch 4/11
3125/3125 - 8s - loss: 8.8656 - accuracy: 0.1005 - 8s/epoch - 3ms/step
Epoch 5/11
3125/3125 - 8s - loss: 8.8655 - accuracy: 0.0961 - 8s/epoch - 3ms/step
Epoch 6/11
3125/3125 - 8s - loss: 8.8652 - accuracy: 0.1009 - 8s/epoch - 3ms/step
Epoch 7/11
3125/3125 - 8s - loss: 8.8653 - accuracy: 0.0978 - 8s/epoch - 3ms/step
Epoch 8/11
3125/3125 - 8s - loss: 8.8648 - accuracy: 0.0991 - 8s/epoch - 3ms/step
Epoch 9/11
3125/3125 - 8s - loss: 8.8649 - accuracy: 0.0998 - 8s/epoch - 3ms/step
Epoch 10/11
3125/3125 - 8s - loss: 8.8647 - accuracy: 0.1008 - 8s/epoch - 3ms/step
Epoch 11/11
3125/3125 - 8s - loss: 8.8647 - accuracy: 0.1000 - 8s/epoch - 3ms/step
313/313 [==============================] - 1s 3ms/step - loss: 8.8643 - accuracy: 0.1000


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/18
3125/3125 - 9s - loss: 2.4227 - accuracy: 0.0999 - 9s/epoch - 3ms/step
Epoch 2/18
3125/3125 - 8s - loss: 2.4177 - accuracy: 0.1032 - 8s/epoch - 3ms/step
Epoch 3/18
3125/3125 - 8s - loss: 2.4109 - accuracy: 0.0991 - 8s/epoch - 3ms/step
Epoch 4/18
3125/3125 - 8s - loss: 2.4227 - accuracy: 0.0975 - 8s/epoch - 3ms/step
Epoch 5/18
3125/3125 - 9s - loss: 2.4078 - accuracy: 0.0985 - 9s/epoch - 3ms/step
Epoch 6/18
3125/3125 - 8s - loss: 2.4232 - accuracy: 0.0984 - 8s/epoch - 3ms/step
Epoch 7/18
3125/3125 - 8s - loss: 2.4053 - accuracy: 0.1005 - 8s/epoch - 3ms/step
Epoch 8/18
3125/3125 - 8s - loss: 2.4155 - accuracy: 0.1003 - 8s/epoch - 3ms/step
Epoch 9/18
3125/3125 - 8s - loss: 2.4129 - accuracy: 0.1001 - 8s/epoch - 3ms/step
Epoch 10/18
3125/3125 - 8s - loss: 2.4256 - accuracy: 0.1012 - 8s/epoch - 3ms/step
Epoch 11/18
3125/3125 - 8s - loss: 2.4068 - accuracy: 0.0996 - 8s/epoch - 3ms/step
Epoch 12/18
3125/3125 - 8s - loss: 2.4094 - accuracy: 0.0968 - 8s/epoch - 3ms/step
Epoch 13/18
3

In [ ]:


for pop in last_pop:
    print(vars(pop))

{'convulution_kernel_size': 2, 'convulution_act': <ActivationType.relu: 'relu'>, 'pooling_layer_type': <PoolingTypes.avg_pooling: 1>, 'pooling_layer_kernel_size': 3, 'first_dense_layer_neurons': 203, 'first_dense_act': <ActivationType.relu: 'relu'>, 'second_dense_layer_neurons': 96, 'second_dense_act': <ActivationType.softmax: 'softmax'>, 'dropout_rate': 0.5111331121652593, 'optimizer_type': <OptimizerType.sgd: 2>, 'learning_rate': 0.019456694931229843, 'epochs': 6, 'batch_size': 32, 'loss': 0.9676980646133423, 'fitness': 2.0}
{'convulution_kernel_size': 2, 'convulution_act': <ActivationType.relu: 'relu'>, 'pooling_layer_type': <PoolingTypes.avg_pooling: 1>, 'pooling_layer_kernel_size': 3, 'first_dense_layer_neurons': 152, 'first_dense_act': <ActivationType.relu: 'relu'>, 'second_dense_layer_neurons': 76, 'second_dense_act': <ActivationType.softmax: 'softmax'>, 'dropout_rate': 0.4948152541730267, 'optimizer_type': <OptimizerType.sgd: 2>, 'learning_rate': 0.019456694931229843, 'epochs':

In [ ]:
print(vars(min(last_pop, key=attrgetter('loss'))))

NameError: ignored

2
